## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Помимо основной целевой части, данный анализ содержит также работы по улучшению датафрейма, так как в ходе работы с ним имелось большое количество пропусков, дубликатов и прочих отклоенении. 
Зависимость выбиралась между такими признаками как наличие или отсутствие детей (а также их количество), цели кредита, уровни дохода и семейного статуса.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 
<a id='Шаг 1. Откройте файл с данными и изучите общую информацию.'></a>

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')

In [2]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

1) df состоит из 12 столбцов, где:float64(2), int64(5), object(5);
2) как минимум в одном столбце данные указаны с разной стилистикой (education)

Вывел (строка)

### Шаг 2. Предобработка данных <a class="anchor" id="step_2"></a>

### Обработка пропусков

In [4]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В ходе анализа было определено, что отсутствует следующее: 
1) имеются пустые строки столбца 'days_employed'
2) имеются пустые строки столбца 'total_income'
Для уровня дохода объективнее было бы заполнить пропуски медианами.
Для количества дней занятости средним.
Как можно заметить, пропуски в этих строках скорее всего зеркальны, что можно проверить следующим образом:

In [5]:
df[(df['days_employed'].isna())&(df['total_income'].isna())].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Изучим наличие нулей в days_employed, таким образом, если нулей не имеется, то можно делать вывод, что имеющиеся пропуски ничто иное, как клиенты которые только начали свою карьеру и открыли счет в банке.

In [6]:
df[df['days_employed']==0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Как и предсказывалось, столбцов со значениями ноль нет.

Как  видно, пропуски не зависят от целей кредита, семейного положения, образования, количества детей и типа доходов. Очевидна взаимосвязь пропусков в двух столбцах количество дней опыта и уровня дохода. Логичней было бы предположить, что пропуски связаны напрямую с 'days_employed'.

Заменим пустые строки столбца 'total_income'. Для этого мы можем разбить всех на разные возрастные группы.

In [7]:
def total_income_na_group(dob):
    if 35 >= dob >= 0:
        return 1
    if 55 >= dob >= 36:
        return 2   
    if 75 >= dob >= 56:
        return 3    

In [8]:
df['dob_id']=df['dob_years'].apply(total_income_na_group)
df.head(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_id
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,2
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,2
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,1


In [9]:
df_mean_income=df.pivot_table(index=['dob_id'], values='total_income', aggfunc='median')
df_mean_income['total_income']=df_mean_income['total_income'].astype('int')
df_mean_income

,total_income
dob_id,
1,147055
2,150625
3,129338


Теперь осталось заменить пропущенные значения 'days_employed' медианой для каждой возрастной группы.

In [10]:
df.loc[(df['dob_id']==1)&(df['days_employed'].isna()), 'total_income']=157391

In [11]:
df.loc[(df['dob_id']==2)&(df['days_employed'].isna()), 'total_income']=161656

In [12]:
df.loc[(df['dob_id']==3)&(df['total_income'].isna()), 'total_income']=140009

In [13]:
df_mean_dob=df.pivot_table(index=['dob_id'], values='days_employed', aggfunc='median')
df_mean_dob['days_employed']=df_mean_dob['days_employed'].astype('int')
df_mean_dob

,days_employed
dob_id,
1,-1197
2,-1760
3,347542


Сделаем замену пропусков в 'days_employed' на медиану возраста.

In [14]:
df.loc[(df['dob_id']==1)&(df['days_employed'].isna()), 'days_employed']=-1197

In [15]:
df.loc[(df['dob_id']==2)&(df['days_employed'].isna()), 'days_employed']=-1760

In [16]:
df.loc[(df['dob_id']==3)&(df['days_employed'].isna()), 'days_employed']=347542

Переведем тип данных 'days_employed' в целочисленный.

In [17]:
df['days_employed']=df['days_employed'].astype('int')
df.head(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_id
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,2
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,2
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,1


Перепроверим наличие пустот во всем датафрейме.

In [18]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dob_id              0
dtype: int64

Данные столбца 'days_employed' в последующем анализе не участвуют, следовательно, весь столбец можно удалить. 

### Вывод

Данные в столбцах 'days_employed' и 'total_income' были заменены на нули. Также была проведена повторная проверка данных на наличие пропусков, что показало отрицательный ответ. 
Пропуски случайны, и обнаружена зависимость между пропусками в столбцах 'days_employed' и 'total_income' - они зеркальны. Причина: вероятно при выгрузке данных были утрачены данные клиентов со значениями столбца 'days_employed' равых нулю. Эти пропуски, возможно, последствия отсчета дней стажа.

### Замена типа данных

Заменим тип данных для:
1) 'total_income'
на integer

In [19]:
df['total_income']=df['total_income'].astype('int')

In [20]:
df.head(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_id
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,2
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1


Уберем с датафрейма столбец 'dob_id' так он нам больше не нужен.

In [21]:
df.drop('dob_id', axis=1, inplace=True)

In [22]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Вывод

Замена оправдана оптимизацией.

### Обработка дубликатов

In [23]:
df.duplicated().sum()

54

Выведем дубликаты. Логично искать дубликаты по всему датафрейму.

In [24]:
df.loc[df.duplicated(), :]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,-1760,41,среднее,1,женат / замужем,0,F,сотрудник,0,161656,покупка жилья для семьи
4182,1,-1197,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,157391,свадьба
4851,0,347542,60,среднее,1,гражданский брак,1,F,пенсионер,0,140009,свадьба
5557,0,347542,58,среднее,1,гражданский брак,1,F,пенсионер,0,140009,сыграть свадьбу
7808,0,347542,57,среднее,1,гражданский брак,1,F,пенсионер,0,140009,на проведение свадьбы
8583,0,347542,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,140009,дополнительное образование
9238,2,-1197,34,среднее,1,женат / замужем,0,F,сотрудник,0,157391,покупка жилья для сдачи
9528,0,347542,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,140009,операции со своей недвижимостью
9627,0,347542,56,среднее,1,женат / замужем,0,F,пенсионер,0,140009,операции со своей недвижимостью
10462,0,347542,62,среднее,1,женат / замужем,0,F,пенсионер,0,140009,покупка коммерческой недвижимости


Попробуем получить какие уникальные значения содержатся в столбцах типа object, чтобы исключить дубликаты с разной стилистикой:
1) education
2) family_status
3) income_type
4) purpose

In [25]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Здесь видны дубликаты в разных регистрах. Приведем их к единому виду.

In [26]:
df['education']=df['education'].str.lower()

In [27]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Дубликатов нет.

In [28]:
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Дубликатов нет.

In [29]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Здесь имеются повторения в разных целях. Ниже, в главе Лемматизация приведем их к единым категориям, но дубликатов в разных регистрах не обнаружено.

Теперь, когда убедились, что дубликаты в разных регистрах есть только в столбце education и привели к единому регистру, проверим наличие дубликатов и ответить на вопрос увеличилось ли их количество. 
Далее удалим строки дубликаты.

In [30]:
df.duplicated().sum()

71

Очевидно, что количество дубликатов увеличилось. Но, необходимо разобраться и с прочими столбцами. 

### Поиск прочих аномалии

Этот раздел назовем "поиском аномалии", для "чистоты" всего датафрейма узнаем какие уникальные данные имеются в столбцах ниже и приведем гипотезы по появлению каждой из них (если таковые будут):
1. "children"
2. "dob_years"
3. "education_id"
4. "gender"
5. "debt"

In [31]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

А вот и аномалии. С 20 все понятно, вероятно при таком большом объеме вводов данных в базу, оператор нажимал кнопку 2 и случайно задевал 0 (Нампад).
А вот -1 это уже что-то интересное. Необходимо узнать сколько этих строк. Если несущественно (тобишь меньше 1% то можно обойтись заменой данных)

In [32]:
round((len(df[df['children']<0])*100/len(df)), 2)

0.22

In [33]:
round((len(df[df['children']==20])*100/len(df)), 2)

0.35

0.22% и 0.35%! Ерунда, скорее всего это всего лишь случаи ошибки. Исправим.

In [34]:
df.loc[df['children']==-1, 'children']=1
df[df['children']==-1].shape

(0, 12)

In [35]:
df.loc[df['children']==20, 'children']=2
df[df['children']==20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Прошерстим остальных.

In [36]:
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

Хоть это и не критично и не требуется по заданию проекта, попробуем узнать, кто эти люди, которым по 0 лет.

In [37]:
df[df['dob_years']==0].shape

(101, 12)

In [38]:
round((len(df[df['dob_years']==0])*100/len(df)), 2)

0.47

101 человек. Это дело требует анализа, сгруппируем весь датафрейм по признаку 'income_type' и выведем медиану возраста каждой категории. Эти медианы и присвоим тем, кто "недавно родились" 

In [39]:
agezero_pivot=df.pivot_table(index=['income_type'], values='dob_years', aggfunc='median')
agezero_pivot

,dob_years
income_type,
безработный,38.0
в декрете,39.0
госслужащий,40.0
компаньон,39.0
пенсионер,60.0
предприниматель,42.5
сотрудник,39.0
студент,22.0


In [40]:
df.loc[(df['dob_years']==0)&(df['income_type']=='студент')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [41]:
df.loc[(df['dob_years']==0)&(df['income_type']=='сотрудник'), 'dob_years']=39
df[(df['dob_years']==0)&(df['income_type']=='студент')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [42]:
#df.loc[(df['dob_years']==0)&(df['income_type']=='сотрудник'), 'dob_years']=39 - пустое множество
df[(df['dob_years']==0)&(df['income_type']=='предприниматель')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [43]:
df.loc[(df['dob_years']==0)&(df['income_type']=='пенсионер'), 'dob_years']=60
df[(df['dob_years']==0)&(df['income_type']=='пенсионер')].shape

(0, 12)

In [44]:
df.loc[(df['dob_years']==0)&(df['income_type']=='компаньон'), 'dob_years']=39
df[(df['dob_years']==0)&(df['income_type']=='компаньон')].shape

(0, 12)

In [45]:
df.loc[(df['dob_years']==0)&(df['income_type']=='госслужащий'), 'dob_years']=40
df[(df['dob_years']==0)&(df['income_type']=='госслужащий')].shape

(0, 12)

In [46]:
#df.loc[(df['dob_years']==0)&(df['income_type']=='в декрете'), 'dob_years']=39 - пустое множество
df[(df['dob_years']==0)&(df['income_type']=='в декрете')].shape

(0, 12)

In [47]:
#df.loc[(df['dob_years']==0)&(df['income_type']=='безработный'), 'dob_years']=39 - пустое множество
df[(df['dob_years']==0)&(df['income_type']=='безработный')].shape

(0, 12)

In [48]:
df['education_id'].unique()

array([0, 1, 2, 3, 4])

In [49]:
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Сколько их XNA? 

In [50]:
df[df['gender']=='XNA'].shape

(1, 12)

Мы можем оставить строку, так как столбец не участвует в дальнейшем анализе. 

In [51]:
df['debt'].unique()

array([0, 1])

In [52]:
df.head(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья


Похоже, все. Проверим повторно на дубликаты.

In [53]:
df.duplicated().sum()

71

С 54 до 71 дубликатов. Сбросим их. Обновим также и индексацию.

In [54]:
df=df.drop_duplicates(keep='first')

In [55]:
df=df.reset_index(drop=True)

### Вывод

Строк дубликатов по всему датафрейму было обнаружено 106. Повторный анализ с учетом регистра не выдал результатов с дубликатами. Сравнивая такое количество строк с количеством строк в самом датафрейме, можно предположить, что эти дубликаты очень похожи на совпадения или же ошибку.

### Лемматизация

Импортируем библиотеку pymystem3

In [56]:
from pymystem3 import Mystem
m=Mystem()

Лемматизуем столбец 'purpose'

In [57]:
def lemmax(row):
    for query in row:
        lemmas = ''.join(m.lemmatize(row))
    return lemmas

In [58]:
df['purpose']=df['purpose'].apply(lemmax)

In [59]:
df['purpose'].isna().sum()

0

In [60]:
df['purpose'].head(10)

0                 покупка жилье\n
1       приобретение автомобиль\n
2                 покупка жилье\n
3    дополнительный образование\n
4               сыграть свадьба\n
5                 покупка жилье\n
6              операция с жилье\n
7                   образование\n
8         на проведение свадьба\n
9       покупка жилье для семья\n
Name: purpose, dtype: object

Теперь, для удобства дальнейшей категоризации (где, забегая вперед, очевидно, что для целей кредита имеются четыре категории 'недвижимость', 'автомобиль', 'образование', 'свадьба') приведем множество с близкими значениями к одной категории заменив значения столбца 'purpose', создадим для этого специальную функцию-подбиратель, но перед этим посчитаем, сколько же таких строк:

In [61]:
def replacer(row):
    for word in row:
        if 'жилье' in row or 'недвижимость' in row:
            return 'недвижимость'
        if 'образование' in row:
            return 'образование'
        if 'автомобиль' in row or 'машина' in row:
            return 'автомобиль'
        if 'свадьба' in row:
            return 'свадьба'
        return 'прочее'

In [62]:
df['purpose'] = df['purpose'].apply(replacer)

In [63]:
df['purpose'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose, dtype: int64

### Вывод

Лемматизация была проведена по столбцу 'purpose'. Данные по столбу были подразделены по категориям.

### Категоризация данных

In [64]:
df.head(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость


Выделим уровни доходов на несколько групп. Для этого применим метод describe. 

In [65]:
df['total_income'].describe().astype('int')

count      21454
mean      166296
std        97888
min        20667
25%       107623
50%       151452
75%       195813
max      2265604
Name: total_income, dtype: int64

In [66]:
df[(df['total_income']>=97851)&(df['total_income']<=107654)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
34,0,-4488,35,среднее,1,женат / замужем,0,F,сотрудник,0,104660,недвижимость
44,1,-1362,26,среднее,1,женат / замужем,0,F,сотрудник,0,100780,автомобиль
45,1,-1039,49,среднее,1,Не женат / не замужем,4,F,сотрудник,0,106385,образование
73,0,-1631,40,среднее,1,гражданский брак,1,M,сотрудник,0,97946,образование
109,0,-2147,59,среднее,1,женат / замужем,0,F,сотрудник,0,107643,образование
...,...,...,...,...,...,...,...,...,...,...,...,...
21346,1,-1577,25,высшее,0,женат / замужем,0,F,сотрудник,0,101065,недвижимость
21368,1,-2737,39,среднее,1,женат / замужем,0,F,сотрудник,0,106459,образование
21382,1,-3584,42,среднее,1,гражданский брак,1,F,компаньон,0,107286,свадьба
21393,1,-949,25,среднее,1,Не женат / не замужем,4,F,сотрудник,0,105262,недвижимость


Определенные значения можно использовать в качестве границ для каждой категории доходов. 

Разделим по четырем категориям:
низкий
средний
высокий
очень высокий

In [67]:
def inc_group(inc):
    if inc <= 97851:
        return 1
    if 97851 <= inc <= 166282:
        return 2   
    if 166282 <= inc <= 195751:
        return 3
    else:
        return 4

In [68]:
df['income_id']=df['total_income'].apply(inc_group)

Составим словарь для каждого уровня дохода

In [69]:
ti_dict={'Уровень':[1, 2, 3, 4]}
tincome_dict=pd.DataFrame(ti_dict, index=['низкий', 'средний', 'высокий', 'очень высокий'])
tincome_dict

,Уровень
низкий,1
средний,2
высокий,3
очень высокий,4


Цели кредита мы уже разделели на 4 категории, теперь предоставим каждой свой id:

In [70]:
def inc_purpose(prp):
    if prp == 'недвижимость':
        return 1
    if prp == 'автомобиль':
        return 2   
    if prp == 'образование':
        return 3
    else:
        return 4 

In [71]:
df['purpose_id']=df['purpose'].apply(inc_purpose)
df.head(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_id,purpose_id
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,4,1
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,2,2
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,2,1


Составим словарь для целей кредита

In [72]:
purpose_dict=df[['purpose', 'purpose_id']].drop_duplicates(keep='first').reset_index(drop=True)
purpose_dict

,purpose,purpose_id
0,недвижимость,1
1,автомобиль,2
2,образование,3
3,свадьба,4


Узнаем какие имеются категории в столбце 'family_status' и составим словарь

In [73]:
df['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [74]:
fs_dict=df[['family_status', 'family_status_id']].drop_duplicates(keep='first').reset_index(drop=True)
fs_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [75]:
def children_yn(cyn):
    if cyn >= 1:
        return 1
    else:
        return 0

In [76]:
df['children_yn']=df['children'].apply(children_yn)

Словарь о статусе наличия/отсутствия детей

In [77]:
c_yn={'Yes':[1], 'No':[0]}
children_dict=pd.DataFrame(c_yn)
children_dict

,Yes,No
0,1,0


### Вывод

Мы выделили словари:
1) по уровню дохода
2) по целям кредита
3) семейному статусу
4) наличию/отсутствию детей
Причины:
1) Это предоставит в дальнейшем удобный вариант отображения данных
2) Целью всего проекта является определение завимости между этими данными и возвратом кредита в срок

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [78]:
df.head(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_id,purpose_id,children_yn
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,4,1,1
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,2,2,1
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,2,1,0


Для поиска данных, которые могут ответить на вопрос имеется ли взаимосвязь между наличием детей и своевременным возвратом кредита выделим две таблицы:
1) сводная таблица по общему числу должников по группам - с детьми и без;
2) 

In [79]:
children_pivot=df.pivot_table(index=['children_yn'], columns='debt', values='total_income', aggfunc='count')
children_pivot

debt,0,1
children_yn,,
0,13028,1063
1,6685,678


In [80]:
children_pivot.loc[0, 'debtor_ratio_%']=round((children_pivot[1][0] / (len(df)))*100, 2)
children_pivot.loc[1, 'debtor_ratio_%']=round((children_pivot[1][1] / (len(df)))*100, 2)
children_pivot.loc[0, 'default_ratio_%']=round((children_pivot[1][0] / (children_pivot[1][0]+children_pivot[0][0]))*100, 2)
children_pivot.loc[1, 'default_ratio_%']=round((children_pivot[1][1] / (children_pivot[1][1]+children_pivot[0][1]))*100, 2)
children_pivot

debt,0,1,debtor_ratio_%,default_ratio_%
children_yn,,,,
0,13028,1063,4.95,7.54
1,6685,678,3.16,9.21


Выведем сводную таблицу по количеству детей.

In [81]:
children_pivot_by_number=df.pivot_table(index=['children'], columns='debt', values='total_income', aggfunc='count')
children_pivot_by_number['debtor_ratio_%']=round((children_pivot_by_number[1]/(len(df))*100), 2)
children_pivot_by_number['default_ratio_%']=round((children_pivot_by_number[1]/(children_pivot_by_number[0]+children_pivot_by_number[1])*100), 2)
children_pivot_by_number

debt,0,1,debtor_ratio_%,default_ratio_%
children,,,,
0,13028.0,1063.0,4.95,7.54
1,4410.0,445.0,2.07,9.17
2,1926.0,202.0,0.94,9.49
3,303.0,27.0,0.13,8.18
4,37.0,4.0,0.02,9.76
5,9.0,NaN,NaN,NaN


In [82]:
children_dict

,Yes,No
0,1,0


### Вывод

Среди всех должников преобладают люди у которых нет детей. Но соотношение по категориям должников к общему числу категории, тобишь процент невозврата преобладает у людей, которые имеют детей (случаи невозврата в категории с детьми преобладает над категорией, в которой детей нет на 23%). 
По сводной таблице, содержащей данные по количеству детей можно узнать о тренде:
С возврастанием количества детей, людей меньше интересуют кредиты либо кредиты - нпозволительный финансовый инструмент в их жизни, что подтверждает процент невозврата, который наоборот имеет тренд роста с ростом количества детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [83]:
family_status_pivot=df.pivot_table(index=['family_status_id'], columns='debt', values='total_income', aggfunc='count')
family_status_pivot

debt,0,1
family_status_id,,
0,11408,931
1,3763,388
2,896,63
3,1110,85
4,2536,274


Узнаем какой процентаж составляет каждая категория и процент невозврата.

In [84]:
family_status_pivot['debtor_ratio_%']=round((family_status_pivot[1]/(len(df))*100), 2)
family_status_pivot['default_ratio_%']=round((family_status_pivot[1]/(family_status_pivot[0]+family_status_pivot[1])*100), 2)
family_status_pivot

debt,0,1,debtor_ratio_%,default_ratio_%
family_status_id,,,,
0,11408,931,4.34,7.55
1,3763,388,1.81,9.35
2,896,63,0.29,6.57
3,1110,85,0.40,7.11
4,2536,274,1.28,9.75


Словарь.

In [85]:
fs_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


### Вывод

Самое преобладающее количество должников в категории людей, которые состоят в браке - 4.34%. При этом процент невозврата среди этой категории также требует особого внимания, занимая третье место среди всех остальных. Очевидно, эта категория заемщиков несут больше риска.

Далее, следуя данным, необходимо выделить тех, кто состоит в гражданском браке, так как они составляют 1.81% от всей клиентской базы, что существенно заметная доля. При этом также процент невозврата у этой же категории ярко выделяется, занимая второе место среди всех остальных.

"Средние" позиции по доле среди всех клиентов занимают те, кто не состоит в браке. Но огромный процент невозврата (9.75% - самый большой среи свсех остальных категории) также требует уделять им больше внимания при выдаче кредитов. 

Последние позиции как среди всех должников, так по покателям коэффициента невохзврата занимают категории "в разводе"  и "вдова/вдовец". Но стоит учесть их количество, что разумно, так как их самое меньшее количество в целом. 
Итого - сравнимо близкие и состоящие в одной категории клиентов с большими рисками это:
1. "в разводе" и "вдовец / вдова"
2. "гражданский брак" и "Не женат / не замужем" 
3. отдельной строкой надо выделить категорию "Не женат / не замужем"

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [86]:
df.head(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_id,purpose_id,children_yn
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,4,1,1
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,2,2,1
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,2,1,0


Составим сводную таблицу, где можно различить зависимость:

In [87]:
income_type_pivot=df.pivot_table(index=['income_id'], columns='debt', values='total_income', aggfunc='count')
income_type_pivot

debt,0,1
income_id,,
1,3873,338
2,8851,822
3,2004,198
4,4985,383


Добавим столбцы указывающие долю среди всех клиентов и процент невозврата для каждой категории.

In [88]:
income_type_pivot['debtor_ratio_%']=round((income_type_pivot[1]/(len(df))*100), 2)
income_type_pivot['default_ratio_%']=round((income_type_pivot[1]/(income_type_pivot[0]+income_type_pivot[1])*100), 2)
income_type_pivot

debt,0,1,debtor_ratio_%,default_ratio_%
income_id,,,,
1,3873,338,1.58,8.03
2,8851,822,3.83,8.50
3,2004,198,0.92,8.99
4,4985,383,1.79,7.13


In [89]:
tincome_dict

,Уровень
низкий,1
средний,2
высокий,3
очень высокий,4


### Вывод

Самые критические показатели у категории со средним уровнем дохода. Их количество составляет почти четыре процента от всей базы клиентов. Незавидное и состояние показателей по невозврату - 8.5%, что уже близко к "бытовому" один из десяти. 
Далее близкие по двум показателям категори - это люди с "низким" и "очень высоким" уровнями дохода.
Люди с доходом от 167К до 196К выделяются среди всех остальных - у данной категории людей большой коэффициент невозврата. Что добавляет их в группу клиентов с большими рисками.


- Как разные цели кредита влияют на его возврат в срок?

In [90]:
purpose_type_pivot=df.pivot_table(index=['purpose_id'], columns='debt', values='total_income', aggfunc='count')
purpose_type_pivot

debt,0,1
purpose_id,,
1,10029,782
2,3903,403
3,3643,370
4,2138,186


Добавим столбцы указывающие долю среди всех клиентов и процент невозврата для каждой категории.

In [91]:
purpose_type_pivot['debtor_ratio_%']=round((purpose_type_pivot[1]/(len(df))*100), 2)
purpose_type_pivot['default_ratio_%']=round((purpose_type_pivot[1]/(purpose_type_pivot[0]+purpose_type_pivot[1])*100), 2)
purpose_type_pivot

debt,0,1,debtor_ratio_%,default_ratio_%
purpose_id,,,,
1,10029,782,3.65,7.23
2,3903,403,1.88,9.36
3,3643,370,1.72,9.22
4,2138,186,0.87,8.00


In [92]:
purpose_dict

,purpose,purpose_id
0,недвижимость,1
1,автомобиль,2
2,образование,3
3,свадьба,4


### Вывод

Самое большое количество невозвратов среди всей базы у людей берущих целевые заимы на приобретение (или для прочих операции) недвижимости. При этом также стоит учесть, что процент невозврата так же заметно высокий и составляет 7.23%.
Далее со схожими показателями выделяются категории людей целью заима которых является приобретение (или для прочих операции) автомобили или получение образования. Следует отметить весьма высокий коэффициент невозврата среди этих двух категории, так как это добавляет негативных оценок к признаку благонадежности.
Люди, намеревающиеся провести свадьбу на приобретенную сумму составляют меньшее количество от общей базы клиентов. Впрочем это не отнимает таких же негативных оценок к признаку благонадежности, так как уровень их невозврата тоже треюует внимания.

### Шаг 4. Общий вывод

In [93]:
children_pivot

debt,0,1,debtor_ratio_%,default_ratio_%
children_yn,,,,
0,13028,1063,4.95,7.54
1,6685,678,3.16,9.21


In [94]:
children_pivot_by_number

debt,0,1,debtor_ratio_%,default_ratio_%
children,,,,
0,13028.0,1063.0,4.95,7.54
1,4410.0,445.0,2.07,9.17
2,1926.0,202.0,0.94,9.49
3,303.0,27.0,0.13,8.18
4,37.0,4.0,0.02,9.76
5,9.0,NaN,NaN,NaN


In [95]:
children_dict

,Yes,No
0,1,0


In [96]:
family_status_pivot

debt,0,1,debtor_ratio_%,default_ratio_%
family_status_id,,,,
0,11408,931,4.34,7.55
1,3763,388,1.81,9.35
2,896,63,0.29,6.57
3,1110,85,0.40,7.11
4,2536,274,1.28,9.75


In [97]:
fs_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [98]:
income_type_pivot

debt,0,1,debtor_ratio_%,default_ratio_%
income_id,,,,
1,3873,338,1.58,8.03
2,8851,822,3.83,8.50
3,2004,198,0.92,8.99
4,4985,383,1.79,7.13


In [99]:
tincome_dict

,Уровень
низкий,1
средний,2
высокий,3
очень высокий,4


In [100]:
purpose_type_pivot

debt,0,1,debtor_ratio_%,default_ratio_%
purpose_id,,,,
1,10029,782,3.65,7.23
2,3903,403,1.88,9.36
3,3643,370,1.72,9.22
4,2138,186,0.87,8.00


In [101]:
purpose_dict

,purpose,purpose_id
0,недвижимость,1
1,автомобиль,2
2,образование,3
3,свадьба,4


Среди всех срезов данных требуется в первую очередь учитывать клиентов без детей. Хотя они и составляют большую часть неблагонадежных должников, но как мы узнали, не все люди с детьми способны себе позволить кредит. А если так, то их коэффициент невозврата имеет тренд роста с ростом количества их детей.
Обратная ситуация идет при анализе семейного статуса заемщиков - овдовевшие берут меньше всех кредитов, но и показатели невозврата также имеют значительный уровень. Неблагонадежными являются те люди, которые состоят в браке. 
Высокий уровня дохода не причина ожидать от клиента своевременного возврата кредита, учитывая показатели невозврата.
Люди приобретающие кредит для покупки автомобиля занимают вторую позицию среди самой неблагонадежных заемщиков по цели заима. При этом у них большой невозврат. Остальные не так сильно отличаются по кожэффициенту невозврата.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения; 
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных; 
- [x]  объяснено, по какому принципу заполнены пропуски; 
- [x]  заменен вещественный тип данных на целочисленный; 
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты; 
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.